# Model for the wine quality prediciton

1. Import data

you can find the source file data from this: [uci datasets](https://archive.ics.uci.edu/dataset/186/wine+quality)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('winequality-red.csv', sep=';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## 2. EDA data

In [3]:
df.iloc[:, :-1].describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000


In [4]:
df['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [5]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

## 3. Preprocessing data

In [7]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [6]:
bins = (2, 6.5, 8)
group_names = ['bad', 'good']
df['quality'] = pd.cut(df['quality'], bins = bins, labels = group_names)

In [8]:
df['quality'].value_counts()

bad     1382
good     217
Name: quality, dtype: int64

In [9]:
encoder = LabelEncoder()
df['quality'] = encoder.fit_transform(df['quality'])

In [10]:
df['quality'].value_counts()

0    1382
1     217
Name: quality, dtype: int64

In [11]:
from imblearn.over_sampling import RandomOverSampler

In [12]:
oversample = RandomOverSampler(sampling_strategy='minority')

X, y = df.drop('quality', axis=1), df['quality']
X, y = oversample.fit_resample(X, y)

In [14]:
y.value_counts()

0    1382
1    1382
Name: quality, dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=0)

X_train.shape, X_test.shape

((1934, 11), (830, 11))

## 4. Build the model

In [15]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [16]:
clf_pipeline = [('scaling', StandardScaler()), 
                ('clf', SVC(random_state = 42))]
pipeline = Pipeline(clf_pipeline)

In [17]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

## 5. The performance of the model

In [20]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [21]:
confusion_matrix(y_test, y_pred)

array([[347,  79],
       [ 39, 365]], dtype=int64)

In [22]:
accuracy_score(y_test, y_pred)

0.8578313253012049

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.81      0.85       426
           1       0.82      0.90      0.86       404

    accuracy                           0.86       830
   macro avg       0.86      0.86      0.86       830
weighted avg       0.86      0.86      0.86       830



## 6. Save the model

In [29]:
from pathlib import Path
import os
import joblib

In [30]:
PATH_SAVE = 'models'
MODEL_NAME = 'svc'

if not os.path.isdir('models'):
    print('create path models')
    os.mkdir(PATH_SAVE)
else:
    print('models path is exist.')

PATH_FILE = Path(PATH_SAVE) / (MODEL_NAME+'.joblib')
PATH_FILE

create path models


WindowsPath('models/svc.joblib')

In [34]:
joblib.dump((pipeline, list(X_test.columns)), PATH_FILE)

['models\\svc.joblib']

## 7. Testing the loaded model

In [36]:
from joblib import load

In [38]:
loaded_model, columns = load('./models/svc.joblib')

In [39]:
columns

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [40]:
loaded_model.predict(X_test)

array([1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,